# Titanic - Machine Learning from Disaster

<img src="https://storage.googleapis.com/kaggle-competitions/kaggle/3136/logos/header.png" width=900>

- Vamos utilizar os [dados disponíveis no Kaggle](https://www.kaggle.com/competitions/titanic)
    - É um dataset de **competição**
    - O resultado é avaliado através da **acurácia**:
        - _"Sua pontuação é a porcentagem de passageiros que você prevê corretamente. Isso é conhecido como acurácia."_

### Importando novamente as bases e fazendo o tratamento dos dados
- Vamos apenas replicar o que fizemos no **[primeiro](https://github.com/lucaslealx/Titanic/blob/main/Parte1.ipynb)**, **[segundo](https://github.com/lucaslealx/Titanic/blob/main/Parte2.ipynb)** e **[terceiro](https://github.com/lucaslealx/Titanic/blob/main/Parte3.ipynb)** arquivo dessa análise (para visualizar os arquivos completos, visite esse link)

In [1]:
# Importando o pandas
import pandas as pd

In [2]:
# Visualizando a base de treino
treino = pd.read_csv('train.csv')
treino.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


In [3]:
# Visualizando a base de teste
teste = pd.read_csv('test.csv')
teste.head(3)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q


- Fazendo o mesmo tratamento inicial que fizemos nas colunas vazias

In [4]:
# Eliminando as colunas com elevada cardinalidade
treino = treino.drop(['Name','Ticket','Cabin'],axis=1)
teste = teste.drop(['Name','Ticket','Cabin'],axis=1)

In [5]:
# Usando a média para substituir valores nulos na coluna de idade
treino.loc[treino.Age.isnull(),'Age'] = treino.Age.mean()
teste.loc[teste.Age.isnull(),'Age'] = teste.Age.mean()

In [6]:
# Tratando a coluna Embarked da base de treino usando a moda 
treino.loc[treino.Embarked.isnull(),'Embarked'] = treino.Embarked.mode()[0]

In [7]:
# E também a coluna Fare da base de teste usando a média
teste.loc[teste.Fare.isnull(),'Fare'] = teste.Fare.mean()

- E fazendo a engenharia de variáveis dos nossos dados

In [8]:
# Usando uma lambda function para tratar a coluna "Sex"
treino['MaleCheck'] = treino.Sex.apply(lambda x: 1 if x == 'male' else 0)
teste['MaleCheck'] = teste.Sex.apply(lambda x: 1 if x == 'male' else 0)

In [9]:
# Fazendo o RobustScaler das colunas Age e Fare
from sklearn.preprocessing import RobustScaler
transformer = RobustScaler().fit(treino[['Age','Fare']])
treino[['Age','Fare']] = transformer.transform(treino[['Age','Fare']])

# e para a base de teste
transformer = RobustScaler().fit(teste[['Age','Fare']])
teste[['Age','Fare']] = transformer.transform(teste[['Age','Fare']])

In [10]:
# Adicionando a coluna sozinho
def sozinho(a,b):
    if (a == 0 and b == 0):
        return 1
    else:
        return 0
    
treino['Sozinho'] = treino.apply(lambda x: sozinho(x.SibSp,x.Parch),axis=1)
teste['Sozinho'] = teste.apply(lambda x: sozinho(x.SibSp,x.Parch),axis=1)

In [11]:
# E criando a coluna de familiares
treino['Familiares'] = treino.SibSp + treino.Parch
teste['Familiares'] = treino.SibSp + treino.Parch

In [12]:
# Fazendo o OrdinalEncoder para a coluna Embarked
from sklearn.preprocessing import OrdinalEncoder
categorias = ['S','C','Q']

enc = OrdinalEncoder(categories=[categorias],dtype='int32')
enc = enc.fit(treino[['Embarked']])
treino['Embarked'] = enc.transform(treino[['Embarked']])

teste['Embarked'] = enc.transform(teste[['Embarked']])

In [13]:
# Apagando as colunas de texto
treino = treino.drop('Sex',axis=1)
teste = teste.drop('Sex',axis=1)

- Visualizando a base resultante

In [14]:
# Visualizando a base de treino
treino.head(3)

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,Embarked,MaleCheck,Sozinho,Familiares
0,1,0,3,-0.592240,1,0,-0.312011,0,1,0,1
1,2,1,1,0.638529,1,0,2.461242,1,0,0,1
2,3,1,3,-0.284548,0,0,-0.282777,0,0,1,0


### Podemos utilizar outros modelos para fazer a previsão

- Podemos selecionar algoritmos diferentes dos que vimos nas partes anteriores (consulte arquivo da **[parte 1](https://github.com/lucaslealx/Titanic/blob/main/Parte1.ipynb)**) considerando os (outros algoritmos disponíveis no scikit-learn](https://scikit-learn.org/stable/supervised_learning.html#supervised-learning)
    - **Regressão Logística**
        - https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression
    - **Random Forest**
        - https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier
    - **MLPClassifier (Redes Neurais)**
        - https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html#sklearn.neural_network.MLPClassifier


- Agora, **além do train_test_split**:
    - https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html
- Vamos usar também o **grid_search** para estimar os melhores parâmetros
    - https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

In [15]:
# Importando o train_test_split
from sklearn.model_selection import train_test_split

In [16]:
# Separando a base de treino em X e y
X = treino.drop(['PassengerId', 'Survived'], axis=1)
y = treino.Survived

In [17]:
# Separando em treino e validação
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.22, random_state=42)

- Para a **Regressão Logística**

In [24]:
# Importando
from sklearn.linear_model import LogisticRegression

In [25]:
# Criando o classificador
clf_rl = LogisticRegression(random_state=42)

In [26]:
# Definindo os parâmetros
parametros_rl = {
    'penalty': ['l1', 'l2'],
    'C': [0.01, 0.1, 1, 10],
    'solver': ['lbfgs', 'liblinear', 'saga'],
    'max_iter': [100, 1000, 5000, 10000]
}

- Para o **Random Forest**

In [27]:
# Importando
from sklearn.ensemble import RandomForestClassifier

In [28]:
# Criando o classificador
clf_rf = RandomForestClassifier(random_state=42)

In [29]:
# Definindo os parâmetros
parametros_rf = {
    'n_estimators': [100, 200, 500, 1000],
    'criterion': ['gini', 'entropy', 'log_loss'],
    'max_depth': [2,4,6,8,None],
    'max_features': ['sqrt', 'log2', None]
}

- E para o **MLPClassifier (Redes Neurais)**

In [30]:
# Importando
from sklearn.neural_network import MLPClassifier

In [31]:
# Criando o classificador
clf_mlp = MLPClassifier(random_state=42)

In [32]:
# Definindo os parâmetros
parametros_mlp = {
    'solver': ['lbfgs', 'sgd', 'adam'],
    'alpha': [10.0**(-1), 10.0**(-5), 10.0**(-7), 10.0**(-10)],
    'max_iter': [200, 500, 1000, 5000]
}

- **Fazendo o grid_search**

In [33]:
# Ignorando os avisos
import warnings
warnings.filterwarnings('ignore')

In [38]:
# Importando o datetime para visualizar a hora atual
from datetime import datetime
def hora_atual():
    agora = datetime.now()
    print(str(agora.hour)+':'+str(agora.minute)+":"+str(agora.second))


In [40]:
# Importando o KFold e o GridSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

In [41]:
# Para a Regressão Logística
hora_atual()
kfold_rl = KFold(shuffle=True, random_state=42, n_splits=8)
grid_search_rl = GridSearchCV(clf_rl, parametros_rl, scoring='accuracy', cv=kfold_rl)
grid_search_rl = grid_search_rl.fit(X_train, y_train)
hora_atual()

16:13:59
16:14:8


In [42]:
# Para o RandomForest
hora_atual()
kfold_rf = KFold(shuffle=True, random_state=42, n_splits=8)
grid_search_rf = GridSearchCV(clf_rf, parametros_rf, scoring='accuracy', cv=kfold_rf)
grid_search_rf = grid_search_rf.fit(X_train, y_train)
hora_atual()

16:15:9
16:35:44


In [43]:
# Para o MLPClassifier
hora_atual()
kfold_mlp = KFold(shuffle=True, random_state=42, n_splits=8)
grid_search_mlp = GridSearchCV(clf_mlp, parametros_mlp, scoring='accuracy', cv=kfold_mlp)
grid_search_mlp = grid_search_mlp.fit(X_train, y_train)
hora_atual()

16:35:48
16:41:25


- **Verificando os melhores parametros**

In [44]:
# Verificando o melhor score da regressão logística
grid_search_rl.best_params_

{'C': 0.1, 'max_iter': 1000, 'penalty': 'l2', 'solver': 'saga'}

In [45]:
# Para o RandomForest
grid_search_rf.best_params_

{'criterion': 'entropy',
 'max_depth': 6,
 'max_features': None,
 'n_estimators': 200}

In [46]:
# e para o MLPClassifier
grid_search_mlp.best_params_

{'alpha': 1e-05, 'max_iter': 1000, 'solver': 'sgd'}

- **E os melhores scores**

In [48]:
# Verificando os melhores parâmetros da regressão logística
grid_search_rl.best_score_

0.8026430098904036

In [49]:
# Para o RandomForest
grid_search_rf.best_score_

0.8285217856188185

In [50]:
# e para o MLPClassifier
grid_search_mlp.best_score_

0.8083400160384924

- **Fazendo a previsão nos dados de validação com cada um dos melhores modelos**

In [52]:
# Para a regressão logística
clf_best_rl = grid_search_rl.best_estimator_
y_pred_rl = clf_best_rl.predict(X_val)

In [53]:
# Para o RandomForest
clf_best_rf = grid_search_rf.best_estimator_
y_pred_rf = clf_best_rf.predict(X_val)

In [54]:
# e para o MLPClassifier
clf_best_mlp = grid_search_mlp.best_estimator_
y_pred_mlp = clf_best_mlp.predict(X_val)

- Vamos novamente **avaliar os modelos**
    - Acurácia (método de avaliação usado na competição):
        - https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html
    - Matriz de confusão (ajuda a visualizar a distribuição dos erros):
        - https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html

- Avaliando a **acurácia**

In [55]:
# Importando
from sklearn.metrics import accuracy_score

In [56]:
# Para a Regressão Logística
accuracy_score(y_val, y_pred_rl)

0.817258883248731

In [57]:
# Para o Random Forest
accuracy_score(y_val, y_pred_rf)

0.817258883248731

In [58]:
# Para o MLPClassifier (Redes Neurais)
accuracy_score(y_val, y_pred_mlp)

0.8020304568527918

- Avaliando a **matriz de confusão**

In [59]:
# Importando
from sklearn.metrics import confusion_matrix

In [60]:
# Para a Regressão Logística
confusion_matrix(y_val, y_pred_rl)

array([[104,  13],
       [ 23,  57]], dtype=int64)

In [61]:
# Para o Random Forest
confusion_matrix(y_val, y_pred_rf)

array([[106,  11],
       [ 25,  55]], dtype=int64)

In [62]:
# Para o MLPClassifier (Redes Neurais)
confusion_matrix(y_val, y_pred_mlp)

array([[101,  16],
       [ 23,  57]], dtype=int64)

### Fazendo a previsão para os dados de teste
- Vamos usar o modelo com melhor precisão para fazer o predict na base de teste

In [63]:
# Visualizando o X_train
X_train.head(3)

,Pclass,Age,SibSp,Parch,Fare,Embarked,MaleCheck,Sozinho,Familiares
800,2,0.330837,0,0,-0.062981,0,1,1,0
781,1,-0.976855,1,0,1.842639,0,0,0,1
542,3,-1.438394,4,2,0.728501,0,0,0,6


In [64]:
# Visualizando a base de teste
teste.head(3)

,PassengerId,Pclass,Age,SibSp,Parch,Fare,Embarked,MaleCheck,Sozinho,Familiares
0,892,3,0.331562,0,0,-0.280670,2,1,1,1
1,893,3,1.311954,1,0,-0.315800,0,0,0,1
2,894,2,2.488424,0,0,-0.201943,2,1,1,0


In [65]:
# Para a base de teste ser igual a base de treino, precisamos eliminar a coluna de id
X_teste = teste.drop('PassengerId',axis=1)

In [66]:
# Utilizando o melhor modelo na base de teste
y_pred = clf_best_rf.predict(X_teste)

In [67]:
# Criando uma nova coluna com a previsão na base de teste
teste['Survived'] = y_pred

In [68]:
# Selecionando apenas a coluna de Id e Survived para fazer o envio
base_envio = teste[['PassengerId','Survived']]

In [69]:
# Exportando para um csv
base_envio.to_csv('resultados4,5.csv',index=False)